In [5]:
# coding:utf-8
"""
本篇为单独调试use_coupon()函数的调试记录，后期将加入UserMainData
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle
import pymysql.cursors

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def easy_print(terms,tname, records=10): #逐条描述前五条记录，方便仔细查看表中值的特征
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname + ' LIMIT 5' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))
            
def full_print(terms,tname): #打出全表所选列，以方便进行统计查看规律，决定处理方法
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname
    cur.execute(sql)
    df = as_pandas(cur)
    print df
    cur.close()
    conn.close()

def merge_table(df1,df2,term1,term2):
    df = pd.merge(df1,df2,how='outer',left_on=term1,right_on=term2)
    return df    
    
def use_coupon():
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    # 1. 用户使用红包数据统计
    sql = 'SELECT customer_id as customerid,deduction_value,order_charge,red_envelop_id FROM km_tbl_use_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)
    # 计算折扣金额总和、业务方账单金额总和
    df1 = df.groupby('customerid',as_index=False).sum()
    df1.columns = ['customerid','use_coupon_sum','use_coupon_order_sum']
    # 计算用户参与红包活动数目
    df2 = df.drop(['deduction_value','order_charge'],axis=1)
    df2['cnt'] = 1
    df2 = df2.groupby(['customerid','red_envelop_id'],as_index=False).sum()
    df2.drop('cnt',axis=1,inplace=True)
    df2 = df2.groupby('customerid',as_index=False).count()
    df2.columns = ['customerid','use_coupon_type_cnt']
    used = pd.merge(df1,df2,how='outer',left_on='customerid',right_on='customerid')
    
    # 2. 对每个用户使用最多红包的场所进行统计
    sql = 'SELECT customer_id as customerid,use_company_id FROM km_tbl_use_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)
    df['cnt'] = 1
    use_com = df.groupby(['customerid','use_company_id'],as_index=False).count()
    use_com.columns = ['customerid','use_company_id','use_company_cnt']
    use_com = use_com.sort_values(['customerid','use_company_cnt'],ascending=[1,0])
    use_com.drop_duplicates(subset='customerid',inplace=True)
    
    # 3. 用户使用红包类型数统计
    sql = 'SELECT customer_id as customerid,coupon_id FROM km_tbl_use_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)    
    df['cnt'] = 1
    df = df.groupby(['customerid','coupon_id'],as_index=False).count()
    use_type = df.groupby('customerid',as_index=False).count()
    use_type.drop('cnt',axis=1,inplace=True)
    use_type.columns = ['customerid','use_coupon_type_cnt']
    
    # 4. 把上面几张表连起来
    df = merge_table(used,use_com,'customerid','customerid')
    df = pd.merge(df,use_type,how='left',left_on=['customerid','use_coupon_type_cnt'],right_on=['customerid','use_coupon_type_cnt'])    
    cur.close()
    conn.close()    
    # 将中间结果暂存于Mysqldb中
    conn = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
    
    df.to_sql(name="ljl_use_coupon", con=conn, flavor='mysql', if_exists='replace', index=False)
    conn.close()

In [6]:
use_coupon()

### order_type这数据不能用

In [11]:
pd.value_counts(df['order_type'])

1    55634
0       42
Name: order_type, dtype: int64

## 1. 用户使用红包数据统计

In [69]:
df.head()

,customerid,use_coupon_sum,use_coupon_order_sum,use_coupon_type_cnt
0,1003913,200,2930,4
1,1006609,15,201,1
2,1018310,20,24,1
3,1019803,400,7042,2
4,1042787,30,32,1


## 2. 对每个用户使用最多红包的场所进行统计

### 使用场所信息

In [78]:
df.head()

,customerid,use_company_id,use_company_cnt
0,1003913,4617,15
1,1006609,4270,1
2,1018310,642,1
3,1019803,4617,20
4,1042787,4984,1


### 进行排序和去重

In [3]:
df.head()

,customerid,use_company_id,use_company_cnt
0,1003913,4617,15
1,1006609,4270,1
2,1018310,642,1
3,1019803,4617,20
4,1042787,4984,1


### 验证每个用户只剩一条记录

In [5]:
pd.value_counts(df['customerid'])

5957969    1
6036584    1
5851360    1
6038525    1
5661153    1
5900763    1
4554736    1
5794584    1
5794585    1
5794586    1
6134035    1
6112423    1
5904172    1
5899932    1
5837096    1
6095873    1
6031917    1
5851363    1
5938944    1
5908571    1
5958506    1
6023829    1
5878666    1
6106120    1
6023826    1
5876263    1
5958501    1
6023822    1
6130539    1
5190754    1
          ..
5809540    1
5772382    1
6012992    1
5878628    1
6064014    1
6066801    1
6067363    1
5794618    1
6125345    1
5867042    1
5992472    1
5902089    1
5436561    1
6134269    1
5911602    1
5951313    1
6123749    1
6143242    1
6134263    1
5900462    1
6130545    1
5953810    1
6055669    1
5841167    1
5955116    1
5841168    1
6111126    1
5834212    1
5907471    1
6136534    1
Name: customerid, dtype: int64

## 3.用户使用红包类型数统计

In [4]:
df.head()

,customerid,use_coupon_type_cnt
0,1003913,5
1,1006609,1
2,1018310,1
3,1019803,5
4,1042787,1


In [5]:
df.shape

(37383, 2)

## 4.把以上表连接起来

In [12]:
df.head()

,customerid,use_coupon_sum,use_coupon_order_sum,use_coupon_type_cnt,use_company_id,use_company_cnt
0,1003913,200,2930,4,4617,15
1,1006609,15,201,1,4270,1
2,1018310,20,24,1,642,1
3,1019803,400,7042,2,4617,20
4,1042787,30,32,1,4984,1


In [13]:
df.shape

(37383, 6)